In [1]:
import bpmeth
import sympy as sp
import re

def format_derivative(expr):
    if isinstance(expr, sp.Derivative):
        # Get the function and its name
        f = expr.expr
        name = f.func.__name__  # e.g., 'a1'
        index_match = re.match(r'([ab]+)([\ds]+)', name)
        if index_match:
           base, idx = index_match.groups()
           prime = "'" * expr.derivative_count
           return sp.var(f"{base}{prime}_{idx}")
        else:
            return expr
    elif isinstance(expr, sp.Function):
        name = expr.func.__name__
        return sp.var(name)
    return expr

def format_function(expr):
    if isinstance(expr, sp.Function):
        name = expr.func.__name__
        index_match = re.match(r'([ab]+)([\ds]+)', name)
        if index_match:
           base, idx = index_match.groups()
           return sp.var(f"{base}_{idx}")        
        return sp.var(name)
    return expr



## Field derivatives expansion with curvature

The expansion is based on this scalar potential definition

$$
\begin{align*}
\phi(x,y,s) &= \sum_{i=0}^{\infty}\phi_i(x,s)\frac{y^i}{i!}\\
\end{align*}
$$
for which the laplace equation gives:

$$
\phi_{i+2} = -\frac{1}{1 + hx} \left( \partial_x \left( (1 + hx) \partial_x \phi_i \right) + \partial_s \left( \frac{1}{1 + hx} \partial_s \phi_i \right) \right)
$$

Defining $\phi_0(x,s)$ and $\phi_1(x,s)$ we get

$$
\begin{align*}
\phi_0(x,s) &= -a_0(s)-\sum_{n=1}^{\infty}a_n(s)\frac{x^n}{n!} &
\phi_1(x,s) &= -\sum_{n=1}^{\infty}b_n(s)\frac{x^{n-1}}{(n-1)!} \\
a_n(s) &= \left. \partial_x^{n-1} B_x(x, y, s) \right|_{x = y = 0} &
b_n(s) &= \left. \partial_x^{n-1} B_y(x, y, s) \right|_{x = y = 0}
a_0(s)
&& a_0'(s)=b_s(s) = B_s(0, 0, s)
\end{align*}
$$

In [2]:
order=2 # in the transverse field derivatives  [n]
nphi=1 # order of the scalar potential components [i]

s,h=sp.var("s,h", real=True)
aa = [sp.Function(f"a{i+1}")(s) for i in range(order)]
bb = [sp.Function(f"b{i+1}")(s) for i in range(order)]
bs = sp.Function("b_s")(s)

fm=bpmeth.FieldExpansion(a=aa,b=bb,bs=bs,hs=h,nphi=nphi)

bxys=fm.get_Bfield(lambdify=False,subs=False)
def simpl(bb):
    bb=bb.series(h,0,2).removeO().expand()
    bb=bb.replace(lambda e: isinstance(e, sp.Derivative),format_derivative)
    bb=bb.replace(lambda e: isinstance(e, sp.Function),format_function)
    return bb
    
    
bx,by,bs=map(simpl,bxys)
bxd=bx.as_coefficients_dict(fm.x,fm.y)
byd=by.as_coefficients_dict(fm.x,fm.y)
bsd=bs.as_coefficients_dict(fm.x,fm.y)
out=[sp.var('term,B_x,B_y,B_s')]
terms=set(bxd)|set(byd)|set(bsd)
key=sp.polys.orderings.monomial_key('grlex', (fm.y, fm.x))
for ll in sorted(terms,key=key):
    out.append([ll,bxd[ll],byd[ll],bsd[ll]])
sp.Matrix(out)

Matrix([
[  term, B_x, B_y,              B_s],
[     1, a_1, b_1,              b_s],
[     x, a_2, b_2,     a'_1 - b_s*h],
[     y, b_2,   0,             b'_1],
[  x**2,   0,   0, -a'_1*h + a'_2/2],
[   x*y,   0,   0,   -b'_1*h + b'_2],
[  x**3,   0,   0,        -a'_2*h/2],
[x**2*y,   0,   0,          -b'_2*h]])

 We have to be careful: limiting in nphi also limits in terms in h!

## Field derivatives expansion without curvature

In [3]:
order=3 # in the transverse field derivatives  [n]
nphi=4 # order of the scalar potential components [i]

s,h=sp.var("s,h", real=True)
aa = [sp.Function(f"a{i+1}")(s) for i in range(order)]
bb = [sp.Function(f"b{i+1}")(s) for i in range(order)]
bs = sp.Function("b_s")(s)

fm=bpmeth.FieldExpansion(a=aa,b=bb,bs=bs,hs="0",nphi=nphi)

bxys=fm.get_Bfield(lambdify=False,subs=False)
def simpl(bb):
    bb=bb.series(h,0,2).removeO().expand()
    bb=bb.replace(lambda e: isinstance(e, sp.Derivative),format_derivative)
    bb=bb.replace(lambda e: isinstance(e, sp.Function),format_function)
    return bb
    
    
bx,by,bs=map(simpl,bxys)
bxd=bx.as_coefficients_dict(fm.x,fm.y)
byd=by.as_coefficients_dict(fm.x,fm.y)
bsd=bs.as_coefficients_dict(fm.x,fm.y)
out=[sp.var('term,B_x,B_y,B_s')]
terms=set(bxd)|set(byd)|set(bsd)
key=sp.polys.orderings.monomial_key('grlex', (fm.y, fm.x))
for ll in sorted(terms,key=key):
    out.append([ll,bxd[ll],byd[ll],bsd[ll]])
sp.Matrix(out)

Matrix([
[     term,              B_x,              B_y,                B_s],
[        1,              a_1,              b_1,                b_s],
[        x,              a_2,              b_2,               a'_1],
[        y,              b_2,      -a_2 - b'_s,               b'_1],
[     x**2,            a_3/2,            b_3/2,             a'_2/2],
[      x*y,              b_3,     -a''_1 - a_3,               b'_2],
[     y**2, -a''_1/2 - a_3/2, -b''_1/2 - b_3/2,  -a'_2/2 - b''_s/2],
[     x**3,                0,                0,             a'_3/6],
[   x**2*y,                0,         -a''_2/2,             b'_3/2],
[   x*y**2,         -a''_2/2,         -b''_2/2, -a'''_1/2 - a'_3/2],
[     y**3,         -b''_2/6,                0, -b'''_1/6 - b'_3/6],
[   x**3*y,                0,         -a''_3/6,                  0],
[x**2*y**2,         -a''_3/4,         -b''_3/4,          -a'''_2/4],
[   x*y**3,         -b''_3/6,                0,          -b'''_2/6],
[x**3*y**2,              

## Fourier series expansion


$$
\Phi(r, \theta, z) = \mathcal{R}e \left\{ \sum_{n=-1}^{\infty} e^{i(n+1)\theta} \sum_{m=0}^{\infty} \mathcal{G}_{n+1,m}(z) r^m \right\}
$$

Using Laplace equation one gets:

$$
\mathcal{G}_{n+1, n+1 + 2k}(z) = \frac{(-1)^k (n+1)!}{2^{2k} (n+1+k)!k!} \mathcal{G}^{[2k]}_{n+1, n+1}(z)
$$

and therefore

$$
B_x(x, y, z) = \mathcal{R}e \left\{ \sum_{n=-1,k=0}^{\infty} \frac{(-1)^k (n+1)!}{2^{2k}(n+1+k)!k!} (x^2 + y^2)^{k-1}(x + iy)^{n+1} \times 
\left[ (n + 1 + 2k)x - i(n + 1)y \right] \mathcal{G}_{n+1}^{[2k]}(z) \right\}
$$

$$
B_y(x, y, z) = \mathcal{I}m \left\{ \sum_{n=-1,k=0}^{\infty} \frac{(-1)^k (n+1)!}{2^{2k}(n+1+k)!k!} (x^2 + y^2)^{k-1}(x + iy)^{n+1} \times 
\left[ -(n + 1)x + i(n + 1 + 2k)y \right] \mathcal{G}_{n+1}^{[2k]}(z) \right\}
$$

$$
B_z(x, y, z) = \mathcal{R}e \left\{ \sum_{n=-1,k=0}^{\infty} \frac{(-1)^k (n+1)!}{2^{2k}(n+1+k)!k!} (x + iy)^{n+1} (x^2 + y^2)^{2k} \mathcal{G}_{n+1}^{[2k+1]}(z) \right\}
$$

We then write
$$ \mathcal{G}_{n} = -i\tilde c_n/n  = (\tilde a_n - i \tilde b_n)/n $$

In [ ]:
import sympy as sp

# Define real symbols
x, y = sp.symbols('x y', real=True)
I = sp.I  # Imaginary unit

# Maximum values for n and k
N_MAX = 3
K_MAX = 1

# Function to generate symbols like K''n or J'''n (declared as real)
def prime_label(base, n_val, order):
    if n_val>0:
        primes = "'" * order
        return sp.Symbol(f"\\tilde {base}_{{{n_val}}}{primes}", real=True)
    else:
        primes = "'" * (order-1)
        return sp.Symbol(f"\\tilde b_{{s}}{primes}", real=True)

# Initialize expressions
Bx_expr = 0
By_expr = 0
Bz_expr = 0

for n_val in range(-1,N_MAX + 1):
    for k_val in range(K_MAX + 1):
        coeff = ((-1)**k_val * sp.factorial(n_val + 1)) / (
            2**(2 * k_val) * sp.factorial(n_val + 1 + k_val) * sp.factorial(k_val)
        )
        if n_val>-1:
            coeff/=(n_val + 1) 
        r2 = x**2 + y**2
        base = (x + I * y)**(n_val + 1)

        A_sym = prime_label("a", n_val + 1, 2 * k_val)
        B_sym = prime_label("b", n_val + 1, 2 * k_val)
        Gk = A_sym - I * B_sym

        # Bx
        bx_vec = (n_val + 1 + 2 * k_val) * x - I * (n_val + 1) * y
        Bx_expr += coeff * r2**(k_val - 1) * base * bx_vec * Gk

        # By
        by_vec = -(n_val + 1) * x + I * (n_val + 1 + 2 * k_val) * y
        By_expr += coeff * r2**(k_val - 1) * base * by_vec * Gk

        # Bz uses G_n^{[2k+1]}(z)
        Az_sym = prime_label("a", n_val + 1, 2 * k_val + 1)
        Bz_sym = prime_label("b", n_val + 1, 2 * k_val + 1)
        Gz = Az_sym - I * Bz_sym
        Bz_expr += coeff * base * r2**k_val * Gz

# Take real/imaginary parts and simplify
Bx_expr = sp.simplify(sp.re(Bx_expr).expand())
By_expr = sp.simplify(sp.im(By_expr).expand())
Bz_expr = sp.simplify(sp.re(Bz_expr).expand())

# Display final results
#print("Bx(x, y, z) =")
#sp.pprint(Bx_expr, use_unicode=True)

#print("\nBy(x, y, z) =")
#sp.pprint(By_expr, use_unicode=True)

#print("\nBz(x, y, z) =")
#sp.pprint(Bz_expr, use_unicode=True)

bx,by,bs=Bx_expr,By_expr,Bz_expr
bxd=bx.as_coefficients_dict(fm.x,fm.y)
byd=by.as_coefficients_dict(fm.x,fm.y)
bsd=bs.as_coefficients_dict(fm.x,fm.y)
out=[sp.var('term,B_x,B_y,B_s')]
terms=set(bxd)|set(byd)|set(bsd)
key=sp.polys.orderings.monomial_key('grlex', (fm.y, fm.x))
for ll in sorted(terms,key=key):
    out.append([ll,bxd[ll],byd[ll],bsd[ll]])
sp.Matrix(out)

Matrix([
[     term,                                B_x,                                B_y,                                   B_s],
[        1,                       \tilde a_{1},                       \tilde b_{1},                          \tilde b_{s}],
[        x,     \tilde a_{2} - \tilde b_{s}'/2,                       \tilde b_{2},                         \tilde a_{1}'],
[        y,                       \tilde b_{2},    -\tilde a_{2} - \tilde b_{s}'/2,                         \tilde b_{1}'],
[     x**2, -3*\tilde a_{1}''/8 + \tilde a_{3},   -\tilde b_{1}''/8 + \tilde b_{3},    \tilde a_{2}'/2 - \tilde b_{s}''/4],
[      x*y, -\tilde b_{1}''/4 + 2*\tilde b_{3}, -\tilde a_{1}''/4 - 2*\tilde a_{3},                         \tilde b_{2}'],
[     y**2,   -\tilde a_{1}''/8 - \tilde a_{3}, -3*\tilde b_{1}''/8 - \tilde b_{3},   -\tilde a_{2}'/2 - \tilde b_{s}''/4],
[     x**3,   -\tilde a_{2}''/6 + \tilde a_{4},  -\tilde b_{2}''/12 + \tilde b_{4},  -\tilde a_{1}'''/8 + \tilde a_{3}'/3],

???? TODO
$$
DFT_\theta[B_x(r,\theta,z)+i B_y(r,\theta,z)]_n=\sum \mathcal{G}_{n}^{[2k]} r^{[2k]}
$$